In [ ]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
import time

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from data import DIV2K
from metric import psnr
from model import srgan

from tensorflow.keras.applications.vgg19 import preprocess_input    
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model

from PIL import Image

%matplotlib inline

In [ ]:
os.makedirs('weights/srgan', exist_ok=True)

### Datasets

In [ ]:
scale = 4
batch_size = 16

In [ ]:
train_x4 = DIV2K(scale=scale, subset='train')
valid_x4 = DIV2K(scale=scale, subset='valid')

In [ ]:
train_x4_ds = train_x4.dataset(batch_size=batch_size)
valid_x4_ds = valid_x4.dataset(batch_size=batch_size)

### Generator pre-training

In [ ]:
generator = srgan.generator()
generator.compile(optimizer=Adam(lr=1e-4), loss='mean_squared_error', metrics=[psnr])
generator.fit(train_x4_ds, epochs=20000)
generator.save_weights(f'weights/srgan/pre_generator.h5')

### Generator fine-tuning (GAN)

In [ ]:
generator = srgan.generator()
generator_optimizer = Adam()
generator.load_weights('weights/srgan/pre_generator.h5')

discriminator = srgan.discriminator()
discriminator_optimizer = Adam()

In [ ]:
mean_squared_error = tf.keras.losses.MeanSquaredError()

vgg = srgan.vgg_54()

@tf.function
def content_loss(hr, sr):
    sr = preprocess_input(sr)
    hr = preprocess_input(hr)
    sr_features = vgg(sr) / 12.75
    hr_features = vgg(hr) / 12.75
    return mean_squared_error(hr_features, sr_features)

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=False)

def generator_loss(sr_out):
    return cross_entropy(tf.ones_like(sr_out), sr_out)

def discriminator_loss(hr_out, sr_out):
    hr_loss = cross_entropy(tf.ones_like(hr_out), hr_out)
    sr_loss = cross_entropy(tf.zeros_like(sr_out), sr_out)
    return hr_loss + sr_loss

In [ ]:
@tf.function
def train_step(lr, hr):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        lr = tf.cast(lr, tf.float32)
        hr = tf.cast(hr, tf.float32)
        
        sr = generator(lr, training=True)

        hr_output = discriminator(hr, training=True)
        sr_output = discriminator(sr, training=True)

        con_loss = content_loss(hr, sr)
        gen_loss = generator_loss(sr_output)
        perc_loss = con_loss + 0.001 * gen_loss
        disc_loss = discriminator_loss(hr_output, sr_output)

    gradients_of_generator = gen_tape.gradient(perc_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    
    return perc_loss, disc_loss

In [ ]:
for learning_rate in [1e-4, 1e-5]:
    generator_optimizer.lr = learning_rate
    discriminator_optimizer.lr = learning_rate
    
    for i in range(2000):
        pls = []
        dls = []
        
        for lr, hr in train_x4_ds:
            pl, dl = train_step(lr, hr)
            pls.append(pl.numpy())
            dls.append(dl.numpy())
        
        print(f'Epoch: {i+1:04d}, lr = {learning_rate:.2e}, perceptual loss = {np.mean(pls):.4f}, discriminator loss = {np.mean(dls):.4f}')

In [ ]:
generator.save_weights(f'weights/srgan/gan_generator.h5')
discriminator.save_weights(f'weights/srgan/gan_discriminator.h5')

### Plot samples

In [ ]:
pre_generator = srgan.generator()
gan_generator = srgan.generator()

pre_generator.load_weights('weights/srgan/pre_generator.h5')
gan_generator.load_weights('weights/srgan/gan_generator.h5')

In [ ]:
def plot_sample(lr_image_path):
    lr = np.expand_dims(Image.open(lr_image_path), axis=0)
    pre_sr = tf.cast(pre_generator.predict(lr), tf.uint8)
    gan_sr = tf.cast(gan_generator.predict(lr), tf.uint8)
    
    plt.figure(figsize=(20, 20))
    
    images = [lr[0], pre_sr[0], gan_sr[0]]
    titles = ['LR', 'SR (PRE)', 'SR (GAN)']
    positions = [1, 3, 4]
    
    for i, (img, title, pos) in enumerate(zip(images, titles, positions)):
        plt.subplot(2, 2, pos)
        plt.imshow(img)
        plt.title(title)
        plt.xticks([])
        plt.yticks([])

In [ ]:
plot_sample('demo/0869x4-crop.png')

In [ ]:
plot_sample('demo/0829x4-crop.png')

In [ ]:
plot_sample('demo/0851x4-crop.png')